In [1]:
import pandas as pd
import numpy as np

In [2]:
# Azure
azure_compute_src = './data/azure.merged.compute.xz.pkl' 
azure_compute_data = pd.read_pickle(azure_compute_src, compression="xz")

In [3]:
# AWS
aws_compute_src = './data/aws.src.compute.xz.pkl' # Kathy, please enter name of amazon data pickle file
aws_compute_data = pd.read_pickle(aws_compute_src, compression="xz")

## Inspect AWS

In [4]:
aws_compute_data.columns

Index(['sku', 'offercode', 'location', 'instance_type', 'instance_family',
       'vcpu', 'clock_speed', 'memory', 'storage', 'network_performance',
       'processor_architecture', 'tenancy', 'os', 'usagetype', 'operation',
       'pd_key', 'price_description', 'unit', 'price'],
      dtype='object')

In [5]:
aws_compute_data['price_description']

0         $0.00 per Dedicated Reservation RHEL m4.10xlar...
1         $0.00 per RHEL with HA and SQL Enterprise i3.8...
2         $0.00 per Reservation RHEL c5a.large Instance ...
3         $7.293 per On Demand Linux with SQL Web inf1.2...
4         $7.234 per Unused Reservation RHEL with SQL We...
                                ...                        
505796    $2.046 per Unused Reservation Windows with SQL...
505797    $0.00 per Dedicated Reservation RHEL with HA a...
505798    $16.704 per On Demand Windows with SQL Server ...
505799    $0.1784 per Unused Reservation RHEL r6g.large ...
505800    $0.511 per Dedicated SUSE i3.xlarge Instance Hour
Name: price_description, Length: 505801, dtype: object

In [6]:
aws_compute_data_trimmed = aws_compute_data.drop(columns=['offercode','processor_architecture','usagetype','operation',
                                                          'pd_key','price_description','unit','network_performance'])

In [7]:
aws_compute_data_trimmed.columns

Index(['sku', 'location', 'instance_type', 'instance_family', 'vcpu',
       'clock_speed', 'memory', 'storage', 'tenancy', 'os', 'price'],
      dtype='object')

In [8]:
# windows only
aws_compute_data_windows = aws_compute_data_trimmed[aws_compute_data_trimmed['os'] == 'Windows']

In [9]:
aws_compute_data_windows.head()

,sku,location,instance_type,instance_family,vcpu,clock_speed,memory,storage,tenancy,os,price
7,FMEJYTPF3M8YDBYD,Asia Pacific (Osaka),r3.8xlarge,Memory optimized,32.0,2.5 GHz,244 GiB,2 x 320 SSD,Host,Windows,0.000
8,B9QWBFQZHCY7X3HM,EU (Ireland),r5ad.2xlarge,Memory optimized,8.0,2.5 GHz,64 GiB,1 x 300 NVMe SSD,Shared,Windows,0.000
11,SA8JPXVV8GR6B4XS,US West (Oregon),g4dn.xlarge,GPU instance,4.0,2.5 GHz,16 GiB,125 GB NVMe SSD,Host,Windows,0.000
18,PGGUDD5Q3K4SP2K5,EU (Ireland),g4ad.4xlarge,GPU instance,16.0,2.8 GHz,64 GiB,600 GB NVMe SSD,Shared,Windows,1.704
19,4CR6D9AX3JUCVGKK,EU (Ireland),m1.large,General purpose,2.0,NaN,7.5 GiB,2 x 420 SSD,Shared,Windows,0.190


## Map Schemas

In [10]:
# Since a smaller set of data, I'm going to use AWS as the base
df_aws = aws_compute_data_windows.drop(columns=['os'])
df_aws['provider'] = 'AWS'

In [11]:
df_aws.columns

Index(['sku', 'location', 'instance_type', 'instance_family', 'vcpu',
       'clock_speed', 'memory', 'storage', 'tenancy', 'price', 'provider'],
      dtype='object')

In [12]:
azure_compute_data.columns

Index(['currencyCode', 'tierMinimumUnits', 'retailPrice', 'unitPrice',
       'armRegionName', 'location', 'effectiveStartDate', 'meterId',
       'meterName', 'productId', 'skuId', 'productName', 'skuName',
       'serviceName', 'serviceId', 'serviceFamily', 'unitOfMeasure', 'type',
       'isPrimaryMeterRegion', 'armSkuName', 'effectiveEndDate',
       'reservationTerm', 'Instance', 'vCPU(s)', 'RAM', 'Temporary storage',
       'Supports Premium SSD Storage', 'Clock', 'Burst', 'Purpose',
       'Single Customer', 'Constrained CPU', 'NVMe Disk', 'GPU', 'GPU Model',
       'NVIDIA GRID Supported', 'Machine Image'],
      dtype='object')

In [13]:
df_azure = azure_compute_data[['Machine Image','location','armSkuName','Purpose','vCPU(s)','Clock',
                                     'RAM','Temporary storage','unitPrice','Single Customer']]

In [14]:
df_azure = df_azure.rename(columns={"Machine Image": "sku", 'armSkuName':'instance_type',
                                                'Purpose':'instance_family','vCPU(s)':'vcpu','Clock':'clock_speed',
                                               'RAM':'memory','Temporary storage':'storage',
                                               'Single Customer':'tenancy', 'unitPrice':'price'})
df_azure['provider'] = 'Azure'

In [15]:
df_azure

,sku,location,instance_type,instance_family,vcpu,clock_speed,memory,storage,price,tenancy,provider
0,DS14 v2,US North Central,Standard_DS14_v2_Promo,Memory,16,2.1,112 GiB,224 GiB,1.482,False,Azure
1,DS14 v2,NO West,Standard_DS14_v2,Memory,16,2.1,112 GiB,224 GiB,2.171,False,Azure
2,DS14 v2,NO West,Standard_DS14_v2,Memory,16,2.1,112 GiB,224 GiB,2.907,False,Azure
3,DS14 v2,CH North,Standard_DS14_v2,Memory,16,2.1,112 GiB,224 GiB,1.824,False,Azure
4,DS14 v2,CH North,Standard_DS14_v2,Memory,16,2.1,112 GiB,224 GiB,9376.000,False,Azure
...,...,...,...,...,...,...,...,...,...,...,...
59058,ND40rs v2,US West 2,Standard_ND40rs_v2,GPU Optimized,40,NaN,672 GiB,"2,900 GiB",22.032,False,Azure
59059,ND40rs v2,US South Central,Standard_ND40rs_v2,GPU Optimized,40,NaN,672 GiB,"2,900 GiB",28.278,False,Azure
59060,ND40rs v2,US South Central,Standard_ND40rs_v2,GPU Optimized,40,NaN,672 GiB,"2,900 GiB",26.438,False,Azure
59061,ND40rs v2,EU West,Standard_ND40rs_v2,GPU Optimized,40,NaN,672 GiB,"2,900 GiB",27.526,False,Azure


## Align Values

In [16]:
def compare_col_vals(df1, df2, col_name):
    d1_u = df1[col_name].unique()
    d2_u = df2[col_name].unique()
    
    for item in d1_u:
        if not any([x in item for x in d2_u]):
            print ("{} mismatched".format(item))


### Instance Family

In [17]:
aws_instances = df_aws['instance_family'].unique()
aws_instances

array(['Memory optimized', 'GPU instance', 'General purpose',
       'Compute optimized', 'Storage optimized', nan, 'Micro instances',
       'FPGA Instances'], dtype=object)

In [18]:
df_aws = df_aws.dropna(subset=['instance_family'])

In [19]:
azure_instances = df_azure['instance_family'].unique()
azure_instances

array(['Memory', 'General', 'High Performance', 'Compute',
       'GPU Optimized', 'Storage Optimized'], dtype=object)

In [20]:
compare_col_vals(df_aws,df_azure,'instance_family')

GPU instance mismatched
Storage optimized mismatched
Micro instances mismatched
FPGA Instances mismatched


In [21]:
#df_azure[df_azure['instance_family'].isnull()] = 'General purpose' # need to reload data
df_aws.loc[df_aws['instance_family']     == 'FPGA Instances'   , 'instance_family'] = 'High Performance'
df_aws.loc[df_aws['instance_family']     == 'GPU instance'     , 'instance_family'] = 'GPU optimized'
df_azure.loc[df_azure['instance_family'] == 'GPU Optimized'    , 'instance_family'] = 'GPU optimized'
df_azure.loc[df_azure['instance_family'] == 'Storage Optimized', 'instance_family'] = 'Storage optimized'
df_azure.loc[df_azure['instance_family'] == 'General'          , 'instance_family'] = 'General purpose'

df_aws = df_aws[df_aws['instance_family'] != 'Micro instances']

In [22]:
compare_col_vals(df_aws,df_azure,'instance_family')

### vcpu

In [23]:
#compare_col_vals(df_aws, df_azure,'vcpu')

col = 'vcpu'
aws_u = df_aws[col].unique()
aws_u

array([ 32.,   8.,   4.,  16.,   2.,  96.,  48.,  40.,  64.,   1.,  12.,
        72.,  36., 448., 128.,  24., 224.])

In [24]:
azu_u = df_azure[col].unique()
azu_u

array(['16', '64', '2', '8', '4 / 16', '8 / 16', '48', '32', '8 / 32',
       '16 / 32', '4 / 8', '2 / 8', '72', '416', '16 / 64', '32 / 64',
       '20', '4', '2 / 4', '1 / 2', '24', '1', '208', '96', '128',
       '1 / 4', '120', '12', '80', '64 / 128', '32 / 128', '6', '44',
       '40'], dtype=object)

### clock_speed

In [25]:
#compare_col_vals(df_aws,df_azure,'clock_speed')

In [26]:
col = 'clock_speed'
aws_u = df_aws[col].unique()
aws_u

array(['2.5 GHz', '2.8 GHz', nan, '3.1 GHz', '2.3 GHz', '2.4 GHz',
       '3 GHz', '4 GHz', 'Up to 3.3 GHz', '4.5 GHz', '2.9 GHz', '2 GHz',
       '2.6 GHz'], dtype=object)

In [27]:
azu_u = df_azure[col].unique()
azu_u

array([2.1 , 2.5 , 2.  ,  nan, 3.2 , 2.6 , 2.35, 3.7 , 2.55])

### memory

In [28]:
compare_col_vals(df_aws,df_azure,'memory')

7.5 GiB mismatched
61 GiB mismatched
30.5 GiB mismatched
15.25 GiB mismatched
21 GiB mismatched
15 GiB mismatched
5.25 GiB mismatched
3.75 GiB mismatched
30 GiB mismatched
976 GiB mismatched
10.5 GiB mismatched
60 GiB mismatched
1 GiB mismatched
0.5 GiB mismatched
60.5 GiB mismatched
17.1 GiB mismatched
24576 GiB mismatched


In [29]:
col = 'memory'
aws_u = df_aws[col].unique()
aws_u

array(['244 GiB', '64 GiB', '16 GiB', '7.5 GiB', '192 GiB', '256 GiB',
       '128 GiB', '384 GiB', '68.4 GiB', '61 GiB', '30.5 GiB', '8 GiB',
       '160 GiB', '15.25 GiB', '122 GiB', '512 GiB', '32 GiB', '1.7 GiB',
       '96 GiB', '4 GiB', '144 GiB', '768 GiB', '732 GiB', '21 GiB',
       '15 GiB', '5.25 GiB', '1952 GiB', '3.75 GiB', '488 GiB',
       '9216 GiB', '48 GiB', '72 GiB', '3904 GiB', '30 GiB', '976 GiB',
       '10.5 GiB', '12288 GiB', '2 GiB', '60 GiB', '1 GiB', '117 GiB',
       '0.5 GiB', '42 GiB', '6144 GiB', '7 GiB', '60.5 GiB', '17.1 GiB',
       '34.2 GiB', '18432 GiB', '24576 GiB'], dtype=object)

In [30]:
azu_u = df_azure[col].unique()
azu_u

array(['112 GiB', '256 GiB', '504 GiB', '28 GiB', '64 GiB', '437.5 GiB',
       '96 GiB', '56 GiB', '16 GiB', '144 GiB', '128 GiB', '192 GiB',
       '5,700 GiB', '432 GiB', '218.75 GiB', '160 GiB', '32 GiB',
       '14 GiB', '4 GiB', '8 GiB', '224 GiB', '2 GiB', '140 GiB',
       '2,850 GiB', '1,750 GiB', '512 GiB', '672 GiB', '384 GiB',
       '2,000 GiB', '7 GiB', '456 GiB', '448 GiB', '11,400 GiB',
       '875 GiB', '1,000 GiB', '110 GiB', '3,800 GiB', '3.5 GiB',
       '440 GiB', '640 GiB', '352 GiB', '900 GiB'], dtype=object)

### storage

In [31]:
col = 'storage'
aws_u = df_aws[col].unique()
aws_u

array(['2 x 320 SSD', '1 x 300 NVMe SSD', '125 GB NVMe SSD',
       '600 GB NVMe SSD', '2 x 420 SSD', '2 x 1900 NVMe SSD',
       '2 x 600 NVMe SSD', 'EBS only', '2 x 900 NVMe SSD', '2 x 840 SSD',
       '1 x 80 SSD', '4 x 600 NVMe SSD', '4 x 900 NVMe SSD',
       '1 x 2500 NVMe SSD', '1 x 160 SSD', '1 x 7500 NVMe SSD',
       '4 x 1900 NVMe SSD', '1 x 1250 NVMe SSD', '1 x 950 NVMe SSD',
       '1 x 150 NVMe SSD', '2 x 2500 NVMe SSD', '2 x 800 SSD',
       '24 x 2000 HDD', '2 x 40 SSD', '1 x 50 NVMe SSD', '1 x 1920 SSD',
       '2 x 80 SSD', '8 x 1900 NVMe SSD', '2 x 300 NVMe SSD',
       '1 x 400 NVMe SSD', '1 x 900 NVMe SSD', '2 x 1920 SSD',
       '1 x 960 SSD', '8 x 7500 NVMe SSD', '225 GB NVMe SSD',
       '1 x 480 SSD', '1 x 475 NVMe SSD', '1 x 1900 NVMe SSD',
       '2 x 900 GB NVMe SSD', '2 x 160 SSD', '2 x 7500 NVMe SSD',
       '1 x 75 NVMe SSD', '1 x 200 NVMe SSD', '4 x 7500 NVMe SSD',
       '900 GB NVMe SSD', '1 x 100 NVMe SSD', '1 x 450 NVMe SSD',
       '3 x 2000 HDD', '

In [32]:
azu_u = df_azure[col].unique()
azu_u

array(['224 GiB', '800 GiB', nan, '384 GiB', '80 GiB', '512 GiB',
       '256 GiB', '1,000 GiB', '112 GiB', '400 GiB', '1,200 GiB',
       '576 GiB', '8,192 GiB', '32 GiB', '50 GiB', '360 GiB', '2,000 GiB',
       '1,600 GiB', '864 GiB', '600 GiB', '64 GiB', '100 GiB', '28 GiB',
       '8 GiB', '16 GiB', '1,440 GiB', '75 GiB', '10 GiB', '280 GiB',
       '200 GiB', '128 GiB', '750 GiB', '4,096 GiB', '150 GiB',
       '1,800 GiB', '2,048 GiB', '40 GiB', '2,400 GiB', '300 GiB',
       '20 GiB', '768 GiB', '56 GiB', '88 GiB', '900 GiB', '678 GiB',
       '2,948 GiB', '672 GiB', '480 GiB', '320 GiB', '14 GiB', '4 GiB',
       '1,024 GiB', '1,344 GiB', '352 GiB', '1,536 GiB', '500 GiB',
       '680 GiB', '700 GiB', '5,630 GiB', '7 GiB', '160 GiB', '640 GiB',
       '1,474 GiB', '736 GiB', '2,807 GiB', '336 GiB', '340 GiB',
       '2,880 GiB', '6,144 GiB', '1,388 GiB', '176 GiB', '3,072 GiB',
       '180 GiB', '6,500 GiB', '2,900 GiB'], dtype=object)

### tenancy

In [33]:
col = 'tenancy'
aws_u = df_aws[col].unique()
aws_u

array(['Host', 'Shared', 'Dedicated'], dtype=object)

In [34]:
azu_u = df_azure[col].unique()
azu_u

array([False,  True])

In [35]:
df_azure.loc[df_azure[col]==False, col] = 'Shared'
df_azure.loc[df_azure[col]==True, col] = 'Dedicated'

In [36]:
azu_u = df_azure[col].unique()
azu_u

array(['Shared', 'Dedicated'], dtype=object)

In [37]:
df_azure[df_azure[col]=='Dedicated']

,sku,location,instance_type,instance_family,vcpu,clock_speed,memory,storage,price,tenancy,provider
1325,G1,EU West,Standard_G1,Memory,2,2.0,28 GiB,384 GiB,0.7700,Dedicated,Azure
1326,G1,EU West,Standard_G1,Memory,2,2.0,28 GiB,384 GiB,0.7000,Dedicated,Azure
1327,G1,US Gov TX,Standard_G1,Memory,2,2.0,28 GiB,384 GiB,0.7625,Dedicated,Azure
1328,G1,US West Central,Standard_G1,Memory,2,2.0,28 GiB,384 GiB,0.6100,Dedicated,Azure
1329,G1,US West Central,Standard_G1,Memory,2,2.0,28 GiB,384 GiB,0.5500,Dedicated,Azure
...,...,...,...,...,...,...,...,...,...,...,...
58389,G4,US West 2,Standard_G4,Memory,16,2.0,224 GiB,"3,072 GiB",3.9220,Dedicated,Azure
58390,G4,US West 2,Standard_G4,Memory,16,2.0,224 GiB,"3,072 GiB",4.4750,Dedicated,Azure
58391,G4,US Gov Virginia,Standard_G4,Memory,16,2.0,224 GiB,"3,072 GiB",4.4000,Dedicated,Azure
58392,G4,JA East,Standard_G4,Memory,16,2.0,224 GiB,"3,072 GiB",4.6020,Dedicated,Azure


### price

In [38]:
col = 'price'
aws_u = df_aws[col].unique()
aws_u

array([ 0.   ,  1.704,  0.19 , ..., 27.289,  0.053, 15.292])

In [39]:
azu_u = df_azure[col].unique()
azu_u

array([1.48200e+00, 2.17100e+00, 2.90700e+00, ..., 1.30885e+05,
       2.82780e+01, 1.18152e+05])

In [40]:
# Good enough..

## Merge AWS and Azure dataframes

In [41]:
df_merged =  df_aws.append(df_azure)
df_merged.reset_index(drop=True)

,sku,location,instance_type,instance_family,vcpu,clock_speed,memory,storage,tenancy,price,provider
0,FMEJYTPF3M8YDBYD,Asia Pacific (Osaka),r3.8xlarge,Memory optimized,32.0,2.5 GHz,244 GiB,2 x 320 SSD,Host,0.000,AWS
1,B9QWBFQZHCY7X3HM,EU (Ireland),r5ad.2xlarge,Memory optimized,8.0,2.5 GHz,64 GiB,1 x 300 NVMe SSD,Shared,0.000,AWS
2,SA8JPXVV8GR6B4XS,US West (Oregon),g4dn.xlarge,GPU optimized,4.0,2.5 GHz,16 GiB,125 GB NVMe SSD,Host,0.000,AWS
3,PGGUDD5Q3K4SP2K5,EU (Ireland),g4ad.4xlarge,GPU optimized,16.0,2.8 GHz,64 GiB,600 GB NVMe SSD,Shared,1.704,AWS
4,4CR6D9AX3JUCVGKK,EU (Ireland),m1.large,General purpose,2.0,NaN,7.5 GiB,2 x 420 SSD,Shared,0.190,AWS
...,...,...,...,...,...,...,...,...,...,...,...
177555,ND40rs v2,US West 2,Standard_ND40rs_v2,GPU optimized,40,NaN,672 GiB,"2,900 GiB",Shared,22.032,Azure
177556,ND40rs v2,US South Central,Standard_ND40rs_v2,GPU optimized,40,NaN,672 GiB,"2,900 GiB",Shared,28.278,Azure
177557,ND40rs v2,US South Central,Standard_ND40rs_v2,GPU optimized,40,NaN,672 GiB,"2,900 GiB",Shared,26.438,Azure
177558,ND40rs v2,EU West,Standard_ND40rs_v2,GPU optimized,40,NaN,672 GiB,"2,900 GiB",Shared,27.526,Azure


In [42]:
df_merged.to_pickle('./data/all.merged.compute.xz.pkl', compression='xz')